# Complete Sentiment Analysis
This requires a complete async sentiment analysis using comprehend.

Ouput's a dataframe the same as objects, with objects that have been analysed by comprehend.
The dataframe will have additional columns: aws_input_line_number, language_sentiment, language_sentiment_scores_*.
`aws_input_line_numbers`: the line number that the object had for the input file.
`language_sentiment`: `POSTIVE|NEUTRAL|NEGATIVE`, sentiment as given by Comprehend.
`language_sentiment_score_[mixed|neutral|positive|negative]`: Score for each class as probability according to AWS comprehend.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime
import pprint

import pandas as pd
import tentaclio

from phoenix.common import artifacts, run_datetime
from phoenix.common import utils
from phoenix.tag.third_party_models.aws_async import complete_sentiment, complete_sentiment_run_params, info_sentiment
from phoenix.tag.third_party_models.aws_async import jobs, language_sentiment_dataset

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parameters
# See phoenix/common/run_datetime.py expected format of parameter
RUN_DATETIME = None

# See phoenix/common/artifacts/registry_environment.py expected format of parameter
ARTIFACTS_ENVIRONMENT_KEY = "local"

# Filters for batch
YEAR_FILTER = 2021
# Without zero padding
MONTH_FILTER = 8
OBJECT_TYPE = "facebook_posts"

In [ ]:
run_params = complete_sentiment_run_params.create(
    ARTIFACTS_ENVIRONMENT_KEY,
    TENANT_ID,
    RUN_DATETIME,
    OBJECT_TYPE,
    YEAR_FILTER,
    MONTH_FILTER,
)

In [ ]:
# Display params.
pprint.pprint(run_params)

In [ ]:
async_job_group = jobs.get_json(run_params.urls.async_job_group)
async_job_group

In [ ]:
job_infos = info_sentiment.get_job_infos(async_job_group)
job_infos

In [ ]:
info_sentiment.are_processable_jobs(job_infos)

In [ ]:
language_sentiment_objects = complete_sentiment.complete_sentiment_analysis(async_job_group, job_infos)

In [ ]:
language_sentiment_objects.head()

In [ ]:
language_sentiment_objects.shape

In [ ]:
a = language_sentiment_dataset.persist(run_params.urls.language_sentiment_objects, language_sentiment_objects, run_params.general.run_dt)